In [3]:
from surprise import NMF, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
import sigopt
conn = sigopt.Connection(client_token='WYKGPMSPGMXTRNJMHSNLQYXOYEJMBUXTYQDZVAKWZNVOTLQO')

# Используем тот же датасет (movielens-100k)
data = Dataset.load_builtin(name='ml-100k', prompt=False)

# Создаем объект алгоритма NMF
algo_nmf = NMF()

# 5-кратная кросс-валидация и вывод результатов
cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Случайное разделение датасета на обучающий и тестовый наборы
# Тестовый набор составляет 25% от всех рейтингов
trainset_nmf, testset_nmf = train_test_split(data, test_size=.25)

# Обучаем алгоритм на обучающем наборе и делаем предсказания для тестового
algo_nmf.fit(trainset_nmf)
predictions_nmf = algo_nmf.test(testset_nmf)

# Вычисляем RMSE
print("RMSE for NMF:", accuracy.rmse(predictions_nmf))

# Оптимизация параметров с использованием GridSearchCV
param_grid_nmf = {'n_epochs': [5, 10], 'lr_bu': [0.002, 0.005],
                  'lr_bi': [0.002, 0.005], 'reg_pu': [0.1, 0.2],
                  'reg_qi': [0.1, 0.2]}
gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=3)
gs_nmf.fit(data)

# Лучший результат RMSE
print("Best RMSE score for NMF:", gs_nmf.best_score['rmse'])

# Комбинация параметров, давшая лучший результат RMSE
print("Best parameters for NMF:", gs_nmf.best_params['rmse'])

# Создаем эксперимент в SigOpt для оптимизации параметров
experiment_nmf = conn.experiments().create(
    name='NMF Recommender',
    parameters=[
        dict(name='n_epochs', type='int', bounds=dict(min=5, max=10)),
        dict(name='lr_bu', type='double', bounds=dict(min=0.002, max=0.005)),
        dict(name='lr_bi', type='double', bounds=dict(min=0.002, max=0.005)),
        dict(name='reg_pu', type='double', bounds=dict(min=0.1, max=0.2)),
        dict(name='reg_qi', type='double', bounds=dict(min=0.1, max=0.2))
    ],
    metrics=[
        dict(name='RMSE', objective='minimize', strategy='optimize'),
        dict(name='MAE', objective='minimize', strategy='store')
    ],
    parallel_bandwidth=1,
    observation_budget=50,
)

print("Created experiment for NMF: https://app.sigopt.com/experiment/" + experiment_nmf.id)

# Функции создания и оценки модели для SigOpt
def create_model_nmf(assignments):
    algo_nmf_opt = NMF(
        n_epochs=assignments['n_epochs'],
        lr_bu=assignments['lr_bu'],
        lr_bi=assignments['lr_bi'],
        reg_pu=assignments['reg_pu'],
        reg_qi=assignments['reg_qi']
    ).fit(trainset_nmf)
    return algo_nmf_opt

def evaluate_model_nmf(assignments):
    algo_nmf_opt = create_model_nmf(assignments)
    predictions_nmf_opt = algo_nmf_opt.test(testset_nmf)
    # Вычисление RMSE и MAE
    return [
        dict(name="RMSE", value=accuracy.rmse(predictions_nmf_opt)),
        dict(name="MAE", value=accuracy.mae(predictions_nmf_opt))
    ]

# Запуск оптимизации параметров в SigOpt
while experiment_nmf.progress.observation_count < experiment_nmf.observation_budget:
    suggestion_nmf = conn.experiments(experiment_nmf.id).suggestions().create()
    value_dicts_nmf = evaluate_model_nmf(suggestion_nmf.assignments)
    conn.experiments(experiment_nmf.id).observations().create(
        suggestion=suggestion_nmf.id,
        values=value_dicts_nmf,
    )

    # Обновление объекта эксперимента
    experiment_nmf = conn.experiments(experiment_nmf.id).fetch()

# Получение лучших параметров и исследование вашего эксперимента
all_best_assignments_nmf = conn.experiments(experiment_nmf.id).best_assignments().fetch()
# Возвращает список объектов, похожих на словари Observation
best_assignments_nmf = all_best_assignments_nmf.data[0].assignments
print("Best Assignments for NMF: " + str(best_assignments_nmf))
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment_nmf.id + "/analysis")


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9661  0.9681  0.9634  0.9624  0.9571  0.9634  0.0037  
MAE (testset)     0.7600  0.7585  0.7578  0.7600  0.7522  0.7577  0.0029  
Fit time          2.27    2.28    2.28    2.60    2.29    2.34    0.13    
Test time         0.30    0.24    0.19    0.34    0.13    0.24    0.07    
RMSE: 0.9644
RMSE for NMF: 0.9643606069618159
Best RMSE score for NMF: 0.9655236786145637
Best parameters for NMF: {'n_epochs': 10, 'lr_bu': 0.005, 'lr_bi': 0.002, 'reg_pu': 0.2, 'reg_qi': 0.2}
Created experiment for NMF: https://app.sigopt.com/experiment/1024558
RMSE: 1.0660
MAE:  0.8763
RMSE: 1.0239
MAE:  0.8347
RMSE: 0.9725
MAE:  0.7579
RMSE: 0.9621
MAE:  0.7532
RMSE: 1.0503
MAE:  0.8609
RMSE: 1.0476
MAE:  0.8583
RMSE: 1.0345
MAE:  0.8462
RMSE: 1.0538
MAE:  0.8642
RMSE: 1.0301
MAE:  0.8409
RMSE: 1.0554
MAE:  0.8663
RMSE: 0.9664
MAE:  0.7557
RMSE: 0.9590
MAE:  0.

c:\Users\grinu\AppData\Local\Programs\Python\Python38\lib\site-packages\sigopt\objects.py:338: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(
